In [14]:
import torch
from typing import Tuple
import pandas as pd

In [20]:
def sgd_factorise_ad(A: torch.Tensor, rank: int, num_epochs=1000, lr=0.01) \
    -> Tuple[torch.Tensor, torch.Tensor]:
    
    m, n = A.shape
    U = torch.rand(m, rank, requires_grad=True)
    V = torch.rand(n, rank, requires_grad=True)
    for epoch in range(0, num_epochs):
        
        U.grad = None
        V.grad = None
        
        error  = torch.nn.functional.mse_loss(U@V.t(), A, reduction='sum')
        error.backward()
        
        u = U.data - U.grad*lr
        U.data = u
        
        v = V.data - V.grad*lr
        V.data = v
        
    return U, V

In [21]:
def truncatedSVD(A: torch.Tensor):

    U, S, V = torch.svd(A)
    m = S.shape[0] -1
    S[m] = 0
    return  U, S, V

In [22]:
original = torch.tensor([[0.3374, 0.6005, 0.1735],
                         [3.3359, 0.0492, 1.8374],
                         [2.9407, 0.5301, 2.2620]])

U, V = sgd_factorise_ad(original, 2)
loss = torch.nn.functional.mse_loss(U@V.t(), original, reduction='sum')
print(f"Approximation {U@V.t()}")
print(f'Loss  is {loss}')\

Approximation tensor([[ 0.2245,  0.5212,  0.3592],
        [ 3.2530, -0.0090,  1.9737],
        [ 3.0378,  0.5983,  2.1023]], grad_fn=<MmBackward>)
Loss  is 0.12191082537174225


In [24]:
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases'
                 + '/iris/iris.data', header=None)
data = torch.tensor(df.iloc[:, [0,1,2,3]].values)
data = data - data.mean(dim=0) # centre the data 

U, V = sgd_factorise_ad(data, 2)
loss = torch.nn.functional.mse_loss(U@V.t(), data, reduction='sum')
print(f"Approximation {U@V.t()}")
print(f'Loss  is {loss}')\


RuntimeError: expected dtype Double but got dtype Float